# (연구&재인) MBTI – 추가자료분석 (all)

신록예찬  
2023-12-13

# 1. Imports

In [1]:
import pandas as pd
import sklearn.model_selection
import sklearn.metrics
from autogluon.multimodal import MultiModalPredictor
from autogluon.tabular import TabularPredictor

/root/anaconda3/envs/ag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

# 2. Data

In [5]:
df = pd.read_csv('mbti_1.csv')
df_train, df_test = sklearn.model_selection.train_test_split(df,test_size=0.5,random_state=42)

In [6]:
labels = list(set(df_train.type))

In [7]:
df_additinal = pd.read_csv('MBTI.csv')
df_additinal2= df_additinal.copy()[429:]

In [112]:
df_additinal2['type'].value_counts()

type
ESTJ    372
ESFJ    194
ENFJ    164
ESFP     91
ESTP     57
ISFJ     32
ENFP      3
Name: count, dtype: int64

In [114]:
for i,post in enumerate(df_additinal['posts']):
    for label in labels:
        if label in post: 
            df_additinal['posts'][i] = post.replace(label,'')

In [115]:
df_train2 = pd.concat([df_train,df_additinal],axis=0).reset_index(drop=True)

In [116]:
df_train2

5679 rows × 2 columns

# 3. Tabular

In [117]:
predictor = TabularPredictor(label='type', eval_metric='acc')
predictor.fit(df_train2)
#predictor = TabularPredictor.load("AutogluonModels/ag-20231214_002525")
predictor.leaderboard()

No path specified. Models will be saved in: "AutogluonModels/ag-20231214_011537"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
    Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
    presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
    presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
    presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
    presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231214_011537"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #140-Ubuntu SMP Thu Aug 4 02:23:37 UTC

In [118]:
predictor.evaluate(df_test)

{'accuracy': 0.6680497925311203,
 'balanced_accuracy': 0.4269238301463123,
 'mcc': 0.6136189980032133}

In [119]:
predictor.predict(df_test)

2802    INTP
2166    INFJ
1919    INTP
360     ENFP
1115    INTJ
        ... 
1415    ENTP
7269    INFP
127     INFJ
3474    INTP
5960    INFJ
Name: type, Length: 4338, dtype: object

In [120]:
model_names = predictor.model_names()
yhats = {model_name:predictor.predict(df_test,model=model_name) for model_name in model_names}

In [121]:
y = df_test['type']

In [122]:
yhats['KNeighborsUnif']

2802    ENFP
2166    INFJ
1919    INTJ
360     INTJ
1115    ENTP
        ... 
1415    INTP
7269    INTJ
127     INFP
3474    ENTP
5960    ENFP
Name: type, Length: 4338, dtype: object

In [123]:
y

2802    INTP
2166    INTJ
1919    INTP
360     ENFP
1115    ENTJ
        ... 
1415    ENFJ
7269    ISFJ
127     INFJ
3474    INTP
5960    INFP
Name: type, Length: 4338, dtype: object

In [124]:
yhats.keys()

dict_keys(['KNeighborsUnif', 'KNeighborsDist', 'NeuralNetFastAI', 'LightGBMXT', 'LightGBM', 'RandomForestGini', 'RandomForestEntr', 'CatBoost', 'ExtraTreesGini', 'ExtraTreesEntr', 'XGBoost', 'NeuralNetTorch', 'LightGBMLarge', 'WeightedEnsemble_L2'])

In [125]:
print(sklearn.metrics.classification_report(y,yhats['LightGBMLarge']))

              precision    recall  f1-score   support

        ENFJ       0.54      0.29      0.38        92
        ENFP       0.64      0.63      0.63       327
        ENTJ       0.69      0.35      0.46       115
        ENTP       0.65      0.64      0.64       337
        ESFJ       0.83      0.25      0.38        20
        ESFP       0.25      0.04      0.07        24
        ESTJ       0.50      0.12      0.20        24
        ESTP       0.61      0.27      0.37        41
        INFJ       0.69      0.70      0.70       745
        INFP       0.66      0.80      0.72       916
        INTJ       0.64      0.67      0.66       523
        INTP       0.68      0.74      0.71       675
        ISFJ       0.81      0.41      0.54        86
        ISFP       0.57      0.50      0.53       130
        ISTJ       0.78      0.44      0.56       113
        ISTP       0.69      0.64      0.66       170

    accuracy                           0.66      4338
   macro avg       0.64   

In [ ]:
              precision    recall  f1-score   support

        ENFJ       0.68      0.35      0.46        92
        ENFP       0.63      0.63      0.63       327
        ENTJ       0.65      0.41      0.50       115
        ENTP       0.63      0.61      0.62       337
        ESFJ       0.67      0.10      0.17        20
        ESFP       0.00      0.00      0.00        24
        ESTJ       0.50      0.08      0.14        24
        ESTP       0.71      0.24      0.36        41
        INFJ       0.70      0.71      0.70       745
        INFP       0.64      0.81      0.72       916
        INTJ       0.65      0.67      0.66       523
        INTP       0.68      0.73      0.70       675
        ISFJ       0.84      0.44      0.58        86
        ISFP       0.59      0.42      0.49       130
        ISTJ       0.71      0.43      0.54       113
        ISTP       0.69      0.63      0.66       170

    accuracy                           0.66      4338
   macro avg       0.62      0.45      0.50      4338
weighted avg       0.66      0.66      0.65      4338
